In [1]:
import numpy as np
import pandas as pd
import openai

In [15]:
# !pip install --upgrade openai
! pip uninstall openai
! pip install openai==0.25.0

Found existing installation: openai 0.26.0
Uninstalling openai-0.26.0:
  Would remove:
    /Users/stepanwork/opt/anaconda3/envs/mlpy38/bin/openai
    /Users/stepanwork/opt/anaconda3/envs/mlpy38/lib/python3.8/site-packages/openai-0.26.0.dist-info/*
    /Users/stepanwork/opt/anaconda3/envs/mlpy38/lib/python3.8/site-packages/openai/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
  Using cached openai-0.25.0-py3-none-any.whl
  Attempting uninstall: openai
    Found existing installation: openai 0.26.0
    Uninstalling openai-0.26.0:
      Successfully uninstalled openai-0.26.0


In [7]:
!openai tools fine_tunes.prepare_data -f ./output/classification/source_title_description.jsonl -q

Analyzing...

- Your file contains 1318 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- All prompts end with suffix `\n\n###\n\n`
- All prompts start with prefix `Source: `

No remediations found.
- [Recommended] Would you like to split into training and validation set? [Y/n]: Y


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified files to `./output/classification/source_title_description_prepared_train.jsonl` and `./output/classification/source_title_description_prepared_valid.jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "./output/classification/source_title_description_prepared_train.jsonl" -v "./output/class

In [ ]:
%env OPENAI_API_KEY=***

In [ ]:
!printenv

# Fine-tuning

In [12]:
! openai api fine_tunes.create -t "./output/classification/source_title_description_prepared_train.jsonl" -v "./output/classification/source_title_description_prepared_valid.jsonl" --compute_classification_metrics --classification_positive_class " no effect END" --n_epochs 2 --model 'ada'

Upload progress: 100%|███████████████████████| 442k/442k [00:00<00:00, 239Mit/s]
Uploaded file from ./output/classification/source_title_description_prepared_train.jsonl: file-7RBRIxtDYQMBseQtpBxJTqnw
Upload progress: 100%|██████████████████████| 113k/113k [00:00<00:00, 76.4Mit/s]
Uploaded file from ./output/classification/source_title_description_prepared_valid.jsonl: file-a7EmIHy1ofMwAFhZ6EdA4X08
Created fine-tune: ft-OyWsY48DgKx4bD08taHKZYFt
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-OyWsY48DgKx4bD08taHKZYFt

/Users/stepanwork/opt/anaconda3/envs/mlpy38/lib/python3.8/site-packages/openai/cli.py:406: RuntimeWarning: coroutine 'FineTune.stream_events' was never awaited
  cls._stream_events(resp["id"])


In [16]:
! openai api fine_tunes.follow -i ft-OyWsY48DgKx4bD08taHKZYFt

[2023-01-10 11:21:56] Created fine-tune: ft-OyWsY48DgKx4bD08taHKZYFt
[2023-01-10 11:23:18] Fine-tune costs $0.07
[2023-01-10 11:23:18] Fine-tune enqueued. Queue number: 0
[2023-01-10 11:23:21] Fine-tune started
[2023-01-10 11:26:52] Completed epoch 1/2
[2023-01-10 11:30:18] Completed epoch 2/2
[2023-01-10 11:30:47] Uploaded model: ada:ft-boost-2023-01-10-16-30-46
[2023-01-10 11:30:50] Uploaded result file: file-MDSzqa3zR0Qv4ZPeCrCf5Zf3
[2023-01-10 11:30:51] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m ada:ft-boost-2023-01-10-16-30-46 -p <YOUR_PROMPT>


In [13]:
! openai api fine_tunes.list

{
  "data": [
    {
      "created_at": 1666055082,
      "fine_tuned_model": "curie:ft-boost-2022-10-18-01-23-20",
      "hyperparams": {
        "batch_size": 2,
        "learning_rate_multiplier": 0.02,
        "n_epochs": 2,
        "prompt_loss_weight": 0.01
      },
      "id": "ft-8s8UatehCjaNod1zmO3UvUv6",
      "model": "curie",
      "object": "fine-tune",
      "organization_id": "org-5kFxGOP5zwvHYnJjOgorapqq",
      "result_files": [
        {
          "bytes": 103336,
          "created_at": 1666056201,
          "filename": "compiled_results.csv",
          "id": "file-R57TMKPWyotf9aN6YUD0Ftrc",
          "object": "file",
          "purpose": "fine-tune-results",
          "status": "processed",
          "status_details": null
        }
      ],
      "status": "succeeded",
      "training_files": [
        {
          "bytes": 360990,
          "created_at": 1666055082,
          "filename": "./output/prepared/named_entity_extraction_3_prepared.jsonl",
          "id":

In [17]:
! openai api fine_tunes.results -i ft-OyWsY48DgKx4bD08taHKZYFt > results.csv

In [18]:
results_data = pd.read_csv('results.csv', header=0)
results_data

,step,elapsed_tokens,elapsed_examples,training_loss,training_sequence_accuracy,training_token_accuracy,validation_loss,validation_sequence_accuracy,validation_token_accuracy,classification/accuracy,classification/precision,classification/recall,classification/auroc,classification/auprc,classification/f1.0
0,1,178,2,0.511862,0.0,0.0,0.603601,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2,420,4,0.337920,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,662,6,0.298063,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,888,8,0.244744,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1098,10,0.181751,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,1051,232038,2102,0.018790,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1051,1052,232248,2104,0.030046,0.5,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1052,1053,232394,2106,0.036238,0.5,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1053,1054,232652,2108,0.025432,0.0,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
results_data[~results_data['classification/accuracy'].isna()]

,step,elapsed_tokens,elapsed_examples,training_loss,training_sequence_accuracy,training_token_accuracy,validation_loss,validation_sequence_accuracy,validation_token_accuracy,classification/accuracy,classification/precision,classification/recall,classification/auroc,classification/auprc,classification/f1.0
527,528,116464,1056,0.030894,0.5,0.8,NaN,NaN,NaN,0.685606,0.682731,0.977011,0.714208,0.824808,0.803783
1054,1055,232894,2110,0.016249,1.0,1.0,NaN,NaN,NaN,0.704545,0.724299,0.890805,0.742656,0.843987,0.798969


In [20]:
openai.api_key = '***'

In [ ]:
ft_model_name = 'ada:ft-boost-2023-01-10-16-30-46'
res = openai.Completion.create(
    model=ft_model_name, 
    prompt="Hunger,Europe,US" + '\n\n###\n\n', 
    max_tokens=256, 
    temperature=0, 
    stop=[" END"]
)
res['choices'][0]['text']

In [11]:
# ! openai api fine_tunes.follow -i ft-8s8UatehCjaNod1zmO3UvUv6

[2022-10-17 21:04:42] Created fine-tune: ft-8s8UatehCjaNod1zmO3UvUv6
[2022-10-17 21:04:54] Fine-tune costs $0.44
[2022-10-17 21:04:54] Fine-tune enqueued. Queue number: 0
[2022-10-17 21:04:55] Fine-tune started
[2022-10-17 21:14:17] Completed epoch 1/2
[2022-10-17 21:22:44] Completed epoch 2/2
[2022-10-17 21:23:21] Uploaded model: curie:ft-boost-2022-10-18-01-23-20
[2022-10-17 21:23:22] Uploaded result file: file-R57TMKPWyotf9aN6YUD0Ftrc
[2022-10-17 21:23:22] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m curie:ft-boost-2022-10-18-01-23-20 -p <YOUR_PROMPT>
